In [1]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus
import requests

In [2]:
url_4583 = f"https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/4583"

In [3]:
resp = requests.get(url_4583, params=None)
resp.raise_for_status()
data_4583 = resp.json()

In [5]:
from collections import defaultdict

# Colección de valores únicos por campo
unique_values = defaultdict(set)

for item in data_4583:
    # Campos del nivel superior
    for key, val in item.items():
        if key != 'Data':
            unique_values[key].add(val)
    # Campos dentro de "Data"
    for entry in item.get('Data', []):
        for key, val in entry.items():
            unique_values[key].add(val)

# Imprimir resultados
for key, vals in unique_values.items():
    print(f"{key}:")
    # Convertimos a lista y ordenamos para legibilidad (si es posible)
    try:
        sorted_vals = sorted(vals)
    except TypeError:
        sorted_vals = list(vals)
    print(sorted_vals)
    print()

COD:
['ECV2441', 'ECV2442', 'ECV2443', 'ECV2444', 'ECV2445', 'ECV2446', 'ECV2447', 'ECV2448', 'ECV2449', 'ECV2450', 'ECV2451', 'ECV2452', 'ECV2453', 'ECV2454', 'ECV2455', 'ECV2456', 'ECV2461', 'ECV2462', 'ECV2463', 'ECV2464', 'ECV2465', 'ECV2466', 'ECV2467', 'ECV2468', 'ECV2469', 'ECV2470', 'ECV2471', 'ECV2472', 'ECV2473', 'ECV2474', 'ECV2475', 'ECV2476', 'ECV2477', 'ECV2478', 'ECV2479', 'ECV2480', 'ECV2485', 'ECV2486', 'ECV2487', 'ECV2488', 'ECV2489', 'ECV2490', 'ECV2491', 'ECV2492', 'ECV2493', 'ECV2494', 'ECV2495', 'ECV2496', 'ECV2497', 'ECV2498', 'ECV2499', 'ECV2500', 'ECV2501', 'ECV2502', 'ECV2503', 'ECV2504', 'ECV2509', 'ECV2510', 'ECV2511', 'ECV2512']

Nombre:
['Ambos sexos. 65 y más años. Total Nacional. Alquiler a precio de mercado. Hogar. ', 'Ambos sexos. 65 y más años. Total Nacional. Alquiler inferior al precio de mercado. Hogar. ', 'Ambos sexos. 65 y más años. Total Nacional. Cesión. Hogar. ', 'Ambos sexos. 65 y más años. Total Nacional. Propiedad. Hogar. ', 'Ambos sexos. D

In [7]:
# Creamos una lista de registros planos
flat = []
for serie in data_4583:
    nombre = serie['Nombre']
    # Solo nos interesan las series de Propiedad de "Ambos sexos"
    if ('Ambos sexos' in nombre) and ('Propiedad. Hogar.' in nombre):
        # Detectamos si es "Total" o "De 16 a 29 años"
        if 'Todas las edades' in nombre:
            grupo = 'total'
        elif 'De 16 a 29 años' in nombre:
            grupo = '16_29'
        else:
            continue

        # Extraemos para cada punto de tiempo
        for punto in serie['Data']:
            flat.append({
                'year': punto['Anyo'],
                'grupo': grupo,
                'pct': punto['Valor']
            })

# Creamos el DataFrame y lo pivotamos
df_4583 = (
    pd.DataFrame(flat)
      .pivot(index='year', columns='grupo', values='pct')
      .reset_index()
      .rename_axis(None, axis=1)
)

In [9]:
df_4583

,year,16_29,total
0,2004,47.7,79.5
1,2005,53.6,80.5
2,2006,56.1,79.5
3,2007,58.1,80.1
4,2008,54.9,79.6
5,2009,50.0,79.3
6,2010,46.9,79.4
7,2011,46.4,79.6
8,2012,42.5,79.2
9,2013,38.5,77.7


In [10]:
df_full = (
    df_4583
    .rename(columns={'total': 'pct_total', '16_29': 'pct_16_29'})
    .sort_values('year')
    .reset_index(drop=True)
)

# Ahora df_full contiene todos los años
print(df_full)

    year  pct_16_29  pct_total
0   2004       47.7       79.5
1   2005       53.6       80.5
2   2006       56.1       79.5
3   2007       58.1       80.1
4   2008       54.9       79.6
5   2009       50.0       79.3
6   2010       46.9       79.4
7   2011       46.4       79.6
8   2012       42.5       79.2
9   2013       38.5       77.7
10  2014       34.2       78.0
11  2015       34.2       77.3
12  2016       28.4       77.1
13  2017       26.5       76.7
14  2018       29.6       76.1
15  2019       25.5       75.9
16  2020       31.8       75.2
17  2021       32.5       75.8
18  2022       30.7       75.9
19  2023       29.0       75.1
20  2024       27.0       73.6


In [11]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]

In [14]:
coll = db['INE_4583_PERFIL_HOGAR']
coll.drop()
coll.insert_many(df_full.to_dict('records'))
print("Registros en la colección: ", coll.count_documents({}))
print("Número de filas de df_full: ", len(df_full))

Registros en la colección:  21
Número de filas de df_full:  21
